In [2]:
import numpy as np
import pandas as pd

In [11]:
dataset = pd.read_excel('dataset.xlsx', sheet_name='LG')
dataset.head()

,GDS Vd=10V [dark],Vgate,ABSID,GDS Vd=10V [1.5w],Vgate.1,ABSID.1,GDS Vd=10V [2.5w],Vgate.2,ABSID.2,GDS Vd=10V [4w],Vgate.3,ABSID.3
0,NaN,-25.0,5.260000e-13,NaN,-25.0,1.243000e-12,NaN,-25.0,1.426000e-12,NaN,-25.0,1.435000e-12
1,NaN,-24.8,3.360000e-13,NaN,-24.8,9.890000e-13,NaN,-24.8,1.140000e-12,NaN,-24.8,1.437000e-12
2,NaN,-24.6,2.820000e-13,NaN,-24.6,1.060000e-12,NaN,-24.6,1.353000e-12,NaN,-24.6,1.267000e-12
3,NaN,-24.4,2.250000e-13,NaN,-24.4,9.910000e-13,NaN,-24.4,1.133000e-12,NaN,-24.4,1.277000e-12
4,NaN,-24.2,3.170000e-13,NaN,-24.2,7.520000e-13,NaN,-24.2,1.158000e-12,NaN,-24.2,1.222000e-12


In [12]:
row, column = dataset.shape
print("Dataset size : ", row , "x" , column)
print("Null Row : ", row + 1)
print("Null column : ", column + 1)

Dataset size :  502 x 12
Null Row :  503
Null column :  13


In [13]:
colList = dataset.columns.tolist() # All columns
nanCol = [] # Declare null columms
nanIndex = []
for i in range(len(colList)):
    nan = ((dataset[colList[i]]).isnull() == True)[0] # nan is True or False
    if(nan): # if nan == true (isnull)
        nanCol.append(colList[i]) 
        nanIndex.append(i)
        
nanIndex.append(column)
print(nanCol)  
print(nanIndex)

# Divide into groups 
for i in range(len(nanCol)):
    nanCol[i] = dataset.iloc[0:row, nanIndex[i]:nanIndex[i+1]]
    
len(nanCol)
nn = nanCol[0].columns[1]
nanCol[0]

['GDS Vd=10V [dark]', 'GDS Vd=10V [1.5w]', 'GDS Vd=10V [2.5w]', 'GDS Vd=10V [4w]']
[0, 3, 6, 9, 12]


,GDS Vd=10V [dark],Vgate,ABSID
0,NaN,-25.0,5.260000e-13
1,NaN,-24.8,3.360000e-13
2,NaN,-24.6,2.820000e-13
3,NaN,-24.4,2.250000e-13
4,NaN,-24.2,3.170000e-13
...,...,...,...
497,NaN,-24.2,1.890000e-13
498,NaN,-24.4,3.880000e-13
499,NaN,-24.6,2.330000e-13
500,NaN,-24.8,3.960000e-13


In [14]:
colName = nanCol[0].columns[1]

value = -24.9 # search value
newY = []

if len( nanCol[0].query("`{0}` == @value".format(colName)) ) > 0:
    for i in range(len(nanCol)) : #幾組   
        ABSIDName = nanCol[i].columns[2] # ABSID || ABSID.1 || ABSID.3 ...
        dd = dataset.query("`{0}` == @value".format(colName))
        for j in range(len(dd)):
            newY.append(dd[ABSIDName].to_numpy()[j]) 
        print(newY)
else :
    for i in range(len(nanCol)) : #幾組
        VgateName = nanCol[i].columns[1] # Vgate || Vgate.1 || Vgate.2 ... 
        ABSIDName = nanCol[i].columns[2] # ABSID || ABSID.1 || ABSID.3 ...
        vgateArr = nanCol[i][VgateName].to_numpy()
        arr = dataset[nanCol[3].columns[1]].to_numpy()

        above = vgateArr[np.searchsorted(arr,value,'left')-1]
        below = vgateArr[np.searchsorted(arr,value,'right')]
        print(above, below) 
        X = value
        x1 = min(above, below)
        x2 =  max(above, below)
        dd = nanCol[i].query("`{0}` == @above or `{0}` == @below".format(VgateName))
        dd = dd.sort_index(ascending=True)
        print(dd)
        for j in range(2) :
            y1 = dd.query("`{0}` == @x1".format(VgateName))[ABSIDName].to_numpy()[j]
            y2 = dd.query("`{0}` == @x2".format(VgateName))[ABSIDName].to_numpy()[j]
            Y = y1 + (X-x1)*(y2-y1)/(x2-x1)
            newY.append(Y)
        print(newY)
            
tmp = []
for k in range(0, len(newY), 2) :
    tmp.append(("[" + nanCol[int(k/2)].columns[0] + "]" , value, newY[k], newY[k+1]))
print(tmp)
tf = pd.DataFrame(columns=['Cate', 'Vgate', 'pos', 'neg'])
for w in range(len(tmp)):
    tf.loc[w] = tmp[w]
print(tf)

# Save to excel
writer = pd.ExcelWriter('result.xlsx', engine='xlsxwriter')
tf.to_excel(writer, sheet_name='Sheet1')
writer.save()

-25.0 -24.8
     GDS Vd=10V [dark]  Vgate         ABSID
0                  NaN  -25.0  5.260000e-13
1                  NaN  -24.8  3.360000e-13
500                NaN  -24.8  3.960000e-13
501                NaN  -25.0  4.180000e-13
[4.309999999999983e-13, 4.069999999999998e-13]
-25.0 -24.8
     GDS Vd=10V [1.5w]  Vgate.1       ABSID.1
0                  NaN    -25.0  1.243000e-12
1                  NaN    -24.8  9.890000e-13
500                NaN    -24.8  9.170000e-13
501                NaN    -25.0  9.750000e-13
[4.309999999999983e-13, 4.069999999999998e-13, 1.1159999999999977e-12, 9.459999999999994e-13]
-25.0 -24.8
     GDS Vd=10V [2.5w]  Vgate.2       ABSID.2
0                  NaN    -25.0  1.426000e-12
1                  NaN    -24.8  1.140000e-12
500                NaN    -24.8  1.132000e-12
501                NaN    -25.0  1.168000e-12
[4.309999999999983e-13, 4.069999999999998e-13, 1.1159999999999977e-12, 9.459999999999994e-13, 1.2829999999999975e-12, 1.1499999999999997e-12]
-